---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 4 - Document Similarity & Topic Modelling

## Part 1 - Document Similarity

For the first part of this assignment, you will complete the functions `doc_to_synsets` and `similarity_score` which will be used by `document_path_similarity` to find the path similarity between two documents.

The following functions are provided:
* **`convert_tag:`** converts the tag given by `nltk.pos_tag` to a tag used by `wordnet.synsets`. You will need to use this function in `doc_to_synsets`.
* **`document_path_similarity:`** computes the symmetrical path similarity between two documents by finding the synsets in each document using `doc_to_synsets`, then computing similarities using `similarity_score`.

You will need to finish writing the following functions:
* **`doc_to_synsets:`** returns a list of synsets in document. This function should first tokenize and part of speech tag the document using `nltk.word_tokenize` and `nltk.pos_tag`. Then it should find each tokens corresponding synset using `wn.synsets(token, wordnet_tag)`. The first synset match should be used. If there is no match, that token is skipped.
* **`similarity_score:`** returns the normalized similarity score of a list of synsets (s1) onto a second list of synsets (s2). For each synset in s1, find the synset in s2 with the largest similarity value. Sum all of the largest similarity values together and normalize this value by dividing it by the number of largest similarity values found. Be careful with data types, which should be floats. Missing values should be ignored.

Once `doc_to_synsets` and `similarity_score` have been completed, submit to the autograder which will run `test_document_path_similarity` to test that these functions are running correctly. 

*Do not modify the functions `convert_tag`, `document_path_similarity`, and `test_document_path_similarity`.*

In [61]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

#바꾸지 말것
def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    text = nltk.word_tokenize(doc)
    pos_tag_list = nltk.pos_tag(text)
    return_list = []
    
    for pos_tag in pos_tag_list:
        if len(wn.synsets(pos_tag[0], convert_tag(pos_tag[1]))) != 0:
            return_list.append(wn.synsets(pos_tag[0], convert_tag(pos_tag[1]))[0])

    
    # Your Code Here
    
    return return_list # Your Answer Here


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    # Your Code Here    
    largest_val = 0
    largest_list = []
    
    for syn1 in s1:
        for syn2 in s2:
             if syn1.path_similarity(syn2) != None:
                    largest_val = max(largest_val,syn1.path_similarity(syn2))
        if largest_val != 0:
            largest_list.append(largest_val)
            
        largest_val = 0
        
            
    return sum(largest_list) / len(largest_list) # Your Answer Here

#바꾸지 말것
def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### test_document_path_similarity

Use this function to check if doc_to_synsets and similarity_score are correct.

*This function should return the similarity score as a float.*

In [ ]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'Use this function to see if your code in doc_to_synsets \
    and similarity_score is correct!'
    return document_path_similarity(doc1, doc2)

<br>
___
`paraphrases` is a DataFrame which contains the following columns: `Quality`, `D1`, and `D2`.

`Quality` is an indicator variable which indicates if the two documents `D1` and `D2` are paraphrases of one another (1 for paraphrase, 0 for not paraphrase).

In [ ]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('paraphrases.csv')
paraphrases.head()

___

### most_similar_docs

Using `document_path_similarity`, find the pair of documents in paraphrases which has the maximum similarity score.

*This function should return a tuple `(D1, D2, similarity_score)`*

In [ ]:
def most_similar_docs():
    
    # Your Code Here
    df = paraphrases[paraphrases["Quality"]==1]
    df["similarity_score"] = df.apply(lambda x : document_path_similarity(x['D1'],x['D2']), axis = 1)
    return_df = df.loc[[df["similarity_score"].idxmax()]]

    return (return_df.iloc[0,1], return_df.iloc[0,2], return_df.iloc[0,3]) # Your Answer Here

In [ ]:
most_similar_docs()

### label_accuracy

Provide labels for the twenty pairs of documents by computing the similarity for each pair using `document_path_similarity`. Let the classifier rule be that if the score is greater than 0.75, label is paraphrase (1), else label is not paraphrase (0). Report accuracy of the classifier using scikit-learn's accuracy_score.

*This function should return a float.*

In [ ]:
def label_accuracy():
    from sklearn.metrics import accuracy_score

    # Your Code Here
    df = paraphrases.copy()
    df["similarity_score"] = df.apply(lambda x : document_path_similarity(x['D1'],x['D2']), axis = 1)
    df["new_rule"] = df["similarity_score"].apply(lambda x : 1 if x > 0.75 else 0)
    
    return accuracy_score(df["Quality"], df["new_rule"]) # Your Answer Here

In [ ]:
label_accuracy()

## Part 2 - Topic Modelling

For the second part of this assignment, you will use Gensim's LDA (Latent Dirichlet Allocation) model to model topics in `newsgroup_data`. You will first need to finish the code in the cell below by using gensim.models.ldamodel.LdaModel constructor to estimate LDA model parameters on the corpus, and save to the variable `ldamodel`. Extract 10 topics using `corpus` and `id_map`, and with `passes=25` and `random_state=34`.

In [1]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [3]:
len(newsgroup_data)

2000

In [4]:
id_map

{0: '000',
 1: '100',
 2: '1990',
 3: '1992',
 4: '1993',
 5: '200',
 6: '2nd',
 7: '300',
 8: '400',
 9: '486',
 10: '500',
 11: '800',
 12: 'ability',
 13: 'able',
 14: 'accept',
 15: 'accepted',
 16: 'access',
 17: 'according',
 18: 'actual',
 19: 'actually',
 20: 'add',
 21: 'addition',
 22: 'additional',
 23: 'address',
 24: 'advance',
 25: 'advice',
 26: 'age',
 27: 'ago',
 28: 'agree',
 29: 'ahead',
 30: 'air',
 31: 'allow',
 32: 'alt',
 33: 'america',
 34: 'american',
 35: 'answer',
 36: 'answers',
 37: 'anybody',
 38: 'apparently',
 39: 'appears',
 40: 'apple',
 41: 'application',
 42: 'applications',
 43: 'appreciate',
 44: 'appreciated',
 45: 'apr',
 46: 'april',
 47: 'area',
 48: 'areas',
 49: 'aren',
 50: 'argument',
 51: 'arguments',
 52: 'article',
 53: 'ask',
 54: 'asked',
 55: 'assume',
 56: 'atheism',
 57: 'auto',
 58: 'available',
 59: 'average',
 60: 'avoid',
 61: 'away',
 62: 'backup',
 63: 'bad',
 64: 'ball',
 65: 'banks',
 66: 'base',
 67: 'baseball',
 68: 'based

In [4]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

# Your code here:
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                           num_topics=NUM_TOPICS,
                                           id2word = id_map,
                                          passes = 10)

### lda_topics

Using `ldamodel`, find a list of the 10 topics and the most significant 10 words in each topic. This should be structured as a list of 10 tuples where each tuple takes on the form:

`(9, '0.068*"space" + 0.036*"nasa" + 0.021*"science" + 0.020*"edu" + 0.019*"data" + 0.017*"shuttle" + 0.015*"launch" + 0.015*"available" + 0.014*"center" + 0.014*"sci"')`

for example.

*This function should return a list of tuples.*

In [32]:
def lda_topics():
    
    # Your Code Here
    topics = ldamodel.print_topics(num_words = 10)
    return_list = []
    
    for topic in topics:
        return_list.append((topic))
        
    return return_list # Your Answer Here

In [33]:
lda_topics()

[(0,
  '0.040*"scsi" + 0.024*"power" + 0.020*"drive" + 0.013*"atheism" + 0.012*"technology" + 0.011*"space" + 0.011*"chip" + 0.010*"device" + 0.009*"alt" + 0.009*"circuit"'),
 (1,
  '0.018*"know" + 0.015*"does" + 0.014*"thanks" + 0.013*"monitor" + 0.013*"just" + 0.013*"card" + 0.013*"cable" + 0.012*"new" + 0.012*"mac" + 0.012*"problem"'),
 (2,
  '0.030*"game" + 0.028*"team" + 0.016*"year" + 0.016*"games" + 0.015*"play" + 0.013*"players" + 0.012*"season" + 0.012*"win" + 0.012*"hockey" + 0.011*"don"'),
 (3,
  '0.064*"edu" + 0.046*"com" + 0.016*"period" + 0.016*"000" + 0.014*"think" + 0.011*"don" + 0.010*"banks" + 0.010*"people" + 0.009*"pitt" + 0.009*"new"'),
 (4,
  '0.019*"just" + 0.019*"people" + 0.018*"don" + 0.014*"good" + 0.013*"problem" + 0.011*"use" + 0.011*"think" + 0.010*"know" + 0.010*"time" + 0.009*"does"'),
 (5,
  '0.035*"space" + 0.023*"nasa" + 0.018*"information" + 0.016*"data" + 0.016*"available" + 0.016*"use" + 0.014*"science" + 0.012*"does" + 0.011*"ftp" + 0.010*"program

### topic_distribution

For the new document `new_doc`, find the topic distribution. Remember to use vect.transform on the the new doc, and Sparse2Corpus to convert the sparse matrix to gensim corpus.

*This function should return a list of tuples, where each tuple is `(#topic, probability)`*

In [5]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [26]:
def topic_distribution():
    
    # Your Code Here
    new_X = vect.transform(new_doc)
    new_corpus = gensim.matutils.Sparse2Corpus(new_X, documents_columns=False)
    return_topics = ldamodel.get_document_topics(new_corpus)
    return_list = []
    test = None
    
    for i in return_topics:
        test = i
    
#     print(test)
        
    return test # Your Answer Here

In [27]:
topic_distribution()

[(0, 0.020004708090284359),
 (1, 0.020000640405055694),
 (2, 0.38743558021294761),
 (3, 0.45253499681523729),
 (4, 0.020004557741566942),
 (5, 0.020003410194152364),
 (6, 0.020004252854730314),
 (7, 0.020004460001192825),
 (8, 0.020004654635613264),
 (9, 0.020002739049219096)]

### topic_names

From the list of the following given topics, assign topic names to the topics you found. If none of these names best matches the topics you found, create a new 1-3 word "title" for the topic.

Topics: Health, Science, Automobiles, Politics, Government, Travel, Computers & IT, Sports, Business, Society & Lifestyle, Religion, Education.

*This function should return a list of 10 strings.*

In [66]:
import re

def topic_names():
    
    # Your Code Here
    topics = lda_topics()
    d1 = []
    d2 = ['Health', 'Science', 'Automobiles', 'Politics', 'Government', 'Travel', 
          'Computers & IT', 'Sports, Business', 'Society & Lifestyle', 'Religion',
          'Education']
    
    for topic in topics:
        d1.append(' '.join(re.findall(r'[a-z]+',topic[1])))
        
    return_list = []
    max_sim = 0
    max_top = ''
    
    for text in d1:
        for topic in d2:
            if max_sim <= document_path_similarity(text,topic):
                max_sim = document_path_similarity(text,topic)
                max_top = topic
        return_list.append((text, max_sim, max_top))
        max_sim = 0
        max_top = None
    
    return_list2 = []
    
    for i in return_list:
        return_list2.append(i[2])
    
    return return_list2 # Your Answer Here

In [67]:
topic_names()

['Computers & IT',
 'Society & Lifestyle',
 'Education',
 'Society & Lifestyle',
 'Education',
 'Science',
 'Automobiles',
 'Education',
 'Automobiles',
 'Society & Lifestyle']